In [2]:
print("OK")

OK


In [3]:
%pwd

'd:\\Medical ChatBot\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'd:\\Medical ChatBot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [7]:
#Extract data from PDF

def load_pdf_file(data):
  loader= DirectoryLoader(data,
                          glob="*.pdf",
                          loader_cls=PyPDFLoader)
  documents= loader.load()
  return documents

In [8]:
extracted_data= load_pdf_file("Data/")

In [13]:
#extracted_data

In [9]:
#split the data into text chunks

def text_split(data):
  text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
  text_chunks= text_splitter.split_documents(extracted_data)
  return text_chunks

In [10]:
text_chunks= text_split(extracted_data)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  5860


In [11]:
#text_chunks

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
#Download the Embedding model from Hugging Face

def download_hugging_face_mode():
  embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-miniLM-L6-v2")
  return embeddings

In [14]:
embeddings = download_hugging_face_mode()

C:\Users\Rony\AppData\Local\Temp\ipykernel_16196\4179058664.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-miniLM-L6-v2")
c:\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [15]:
query_result = embeddings.embed_query("Hello World")
print("Length, ", len(query_result))

Length,  384


In [28]:
#query_result

In [38]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [42]:
from pinecone.grpc import PineconeGRPC as pinecone
from pinecone import ServerlessSpec
from pinecone import Pinecone, ServerlessSpec
import os

pc = pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"

pc.create_index(
  name=index_name,
  dimension=384,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

In [43]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [44]:
#Embed each chunk and upsert the embeddings to the Pinecone index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
  documents=text_chunks,
  embedding=embeddings,
  index_name=index_name
)

In [45]:
#Load Existing Pinecone Index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
  index_name=index_name,
  embedding=embeddings
)

In [46]:
docsearch

In [47]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [48]:
retriever_doc = retriever.invoke("What is Acne?")

In [49]:
retriever_doc

[Document(id='4cf0a3ff-9e41-492c-b24d-1dda4f272a4f', metadata={'page': 39.0, 'source': 'Data\\medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='17620f1a-3171-41fd-b702-8c92e9f886f1', metadata={'page': 38.0, 'source': 'Data\\medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='634fd160-a05c-4d8b-b322-87427cebb937', metadata={'page': 37.0, 'source': 'Data\\medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when

In [50]:
retriever_doc = retriever.invoke("What is Acne?")

In [51]:
retriever_doc

[Document(id='4cf0a3ff-9e41-492c-b24d-1dda4f272a4f', metadata={'page': 39.0, 'source': 'Data\\medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='17620f1a-3171-41fd-b702-8c92e9f886f1', metadata={'page': 38.0, 'source': 'Data\\medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='634fd160-a05c-4d8b-b322-87427cebb937', metadata={'page': 37.0, 'source': 'Data\\medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when

In [69]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [71]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
  "You are an assistant for question-answering tasks. "
  "Use the following pieces of retrieved context to answer "
  "the question. If you don't know the answer, say that you "
  "don't know. Use three sentences maximum and keep the "
  "answer concise."
  "\n\n"
  "{context}"
)


prompt = ChatPromptTemplate.from_messages(
  [
    ("system", system_prompt),
    ("human", "{input}")
  ]
)

In [72]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [73]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])



Acne is a common skin disease that causes pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. Acne vulgaris, the medical term for common acne, is the most common skin disease and affects approximately 17 million people in the United States.


In [74]:
response = rag_chain.invoke({"input": "What is Liver cirrhosis?"})
print(response["answer"])


Liver cirrhosis is a chronic liver disease that is characterized by the invasion of connective tissue and degeneration of proper liver functioning. It can be caused by various factors such as alcoholism, metabolic diseases, syphilis, and congestive heart disease. One of the common symptoms of liver cirrhosis is jaundice, and it can lead to serious complications such as portal hypertension and ascites. Treatment for liver cirrhosis may include lifestyle changes, medication, and in severe cases, a liver transplant.
